In [ ]:
# -*- coding: utf-8 -*-

"""Contains function to perform INIT"""

from __future__ import absolute_import

import six
from optlang.symbolics import Zero


def init(model, expression_profile, epsilon, weights, condition=None):
    """
    Integrative Network Inference for Tissues (INIT)[1]
    
    Parameters
    ----------
    model: cobra.Model
        The constraint-based model to perform INIT on.
    expression_profile: ExpressionProfile
        The expression profile to integrate in the model.
    epsilon: float
        A positive tolerance.
    weights: dict(cobra.Model.id: float)
        The weights to be assigned to the reactions.
    condition: str or int, optional (default None)
        The condition from the expression profile. If None, the first condition is used.
        
    Returns
    -------
    tissue-specific model: cobra.Model
    
    References
    ----------
    .. [1]  Agren R, Bordel S, Mardinoglu A, Pornputtapong N, Nookaew I, et al. (2012)
            Reconstruction of Genome-Scale Active Metabolic Networks for 69 Human Cell
            Types and 16 Cancer Types Using INIT.
            PLoS Comput Biol 8(5): e1002518.
            doi:10.1371/journal.pcbi.1002518
    """
    
    assert isinstance(model, cobra.Model)
    assert isinstance(expression_profile, ExpressionProfile)
    assert isinstance(epsilon, float)
    assert isinstance(weights, dict)
    assert isinstance(condition, (str, int, None))
    
    y_vars = []
    x_vars = []
    consts = []
    
    condition = expression_profile.conditions[0] if condition is None else condition
    reaction_profile = expression_profile.to_reaction_dict(condition, model, not_measured_value, normalization)
    
    with model:
        prob = model.problem
        # Reactions
        for rxn_id, rxn_exp in six.iteritems(reaction_profile):
            rxn = model.reactions.get_by_id(rxn_id)
            y = prob.Variable("y_%s" % rxn_id, type="binary")
            const = prob.Constraint(rxn.flux_expression - (1000 * y),
                                    ub=0.0, name="rxn_const_%s" % rxn_id)
            epsilon_const = prob.Constraint(rxn.flux_expression + (1000 * (1 - y)) - epsilon,
                                            lb=0.0, name="rxn_epsilon_const_%s" % rxn_id)
            y_vars.append(y)

            if rxn.reversibility is False:
                irreversible_rxn_const = prob.Constraint(rxn.flux_expression,
                                                       lb=0.0, name="irrev_rxn_const_%s" %rxn_id)
                consts.extend([y, const, epsilon_const, irreversible_rxn_const])
            else:
                consts.extend([y, const, epsilon_const])
            # Metabolites
            for met in rxn.metabolites:
                x = prob.Variable("x_%s_%s".format(rxn_id, met.id), type="binary")
                # TODO: Should .get_coefficient() be replaced by .get_by_id()?
                met_const = prob.Constraint(rxn.metabolites.get_coefficient(met) - (1000 * x),
                                              ub=0.0, name="met_const_%s_%s".format(rxn_id, met.id))
                met_const_epsilon = prob.Constraint(rxn.metabolites.get_coefficient(met) + (1000 * (1 - x)) - epsilon,
                                                    lb=0.0, name="met_epsilon_const_%s_%s".format(rxn_id, met.id))
                met_coefficient_const = prob.Constraint(rxn.metbolites.get_coefficient(met),
                                                        lb=0.0, name="met_coefficient_const_%s_%s".format(rxn_id, met.id))
                consts.extend([x, met_const, met_const_epsilon, met_coefficient_const])
                x_vars.append(x)
        
        model.add_cons_vars(consts)
        model.objective = prob.Objective(Zero, sloppy=True, direction="max")
        model.objective.set_linear_coefficients({v: weights.get(v) for v in y_vars})
        model.objective.set_linear_coefficients({v: 1.0 for v in x_vars})
        
        sol = model.slim_optimize()
        
        return (model, sol)